In [1]:
#playing around with using Postgres to use data in SQL

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

dbname = 'pets_db' #name for new db
username = 'post-doc' # postgres username

## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

# read a database from CSV and load it into a pandas dataframe
pets_data = pd.read_csv('chicago_adopted_20k_Jan_21.csv', index_col=0)

## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
pets_data.to_sql('pets_data_table', engine, if_exists='replace')





postgres://post-doc@localhost/pets_db
True


In [2]:
#query my data in SQL, return only dogs that are babies

# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM pets_data_table WHERE age='Adult';
"""
pets_data_from_sql = pd.read_sql_query(sql_query,con)
pets_data_from_sql.head()

,index,id,organization_id,url,type,species,age,gender,size,coat,...,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,_links.self.href,_links.type.href,_links.organization.href
0,0,47180520,IL53,https://www.petfinder.com/dog/gwen-47180520/il...,Dog,Dog,Adult,Female,Large,None,...,(847) 705-2653,4927 Main Street,None,Skokie,IL,60077.0,US,/v2/animals/47180520,/v2/types/dog,/v2/organizations/il53
1,1,47171077,IL496,https://www.petfinder.com/dog/keeper-47171077/...,Dog,Dog,Adult,Male,Medium,Medium,...,7085550000,2144 North Wood Street,None,Chicago,IL,60625.0,US,/v2/animals/47171077,/v2/types/dog,/v2/organizations/il496
2,3,47160277,IL922,https://www.petfinder.com/dog/bernie-47160277/...,Dog,Dog,Adult,Male,Medium,None,...,(847) 338-3252,None,None,Island Lake,IL,60042.0,US,/v2/animals/47160277,/v2/types/dog,/v2/organizations/il922
3,5,47159529,IL274,https://www.petfinder.com/dog/rosemary-tea-471...,Dog,Dog,Adult,Female,Medium,None,...,None,None,None,Hillside,IL,60162.0,US,/v2/animals/47159529,/v2/types/dog,/v2/organizations/il274
4,9,47148767,IL782,https://www.petfinder.com/dog/mama-mia-4714876...,Dog,Dog,Adult,Female,Medium,None,...,1-779-206-2132,2303 Oak Leaf Street,None,Joliet,IL,60435.0,US,/v2/animals/47148767,/v2/types/dog,/v2/organizations/il782
